## **1.Implement AlexNet and ResNet-18 Architecture  in PyTorch**

**Implementing AlexNet**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x)) # Removed pooling here
        x = F.relu(self.conv4(x)) # Removed pooling here
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

# Create a random input tensor with shape (batch_size, channels, height, width)
input_tensor = torch.randn(4, 3, 224, 224)  # Use 224x224 as the input size

# Instantiate the model
alexnet_model = AlexNet(num_classes=10)

# Get the model output
output = alexnet_model(input_tensor)

# Output shape
print(output.shape)  # Should print: torch.Size([4, 10])
print(output)  # Prints the raw scores for each class

torch.Size([4, 10])
tensor([[-1.7156e-02,  1.1554e-02, -2.7084e-02,  3.2982e-05,  4.8485e-03,
         -9.7351e-03, -3.7149e-04, -9.2787e-03, -9.4921e-03,  1.2892e-02],
        [-1.6288e-02, -2.8804e-03, -3.0234e-03,  1.6097e-04,  3.7497e-03,
         -8.2696e-03,  8.6850e-03, -2.0397e-02,  9.3941e-03,  2.7659e-03],
        [-2.4368e-02,  9.8528e-03, -1.4246e-02, -1.2146e-02, -7.7797e-03,
         -2.7456e-02, -4.0640e-03, -1.5262e-02, -4.6975e-03,  6.2381e-03],
        [-1.6860e-02,  6.6081e-03, -1.6314e-02, -1.8434e-02,  6.3818e-03,
         -5.5130e-03, -8.9831e-03, -2.0591e-02, -1.4087e-03,  1.1883e-02]],
       grad_fn=<AddmmBackward0>)


**Implementing ResNet-18**

In [2]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

def resnet18(num_classes=1000):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)
# Create a random input tensor with shape (batch_size, channels, height, width)
input_tensor = torch.randn(4, 3, 224, 224)

# Instantiate the model
resnet_model = resnet18(num_classes=10)

# Get the model output
output = resnet_model(input_tensor)

# Output shape
print(output.shape)  # Should print: torch.Size([4, 10])
print(output)  # Prints the raw scores for each class


torch.Size([4, 10])
tensor([[ 0.0122,  0.1111,  0.1089, -0.6872, -0.1790,  0.2567, -0.0155, -0.0827,
          0.0224, -0.3780],
        [-0.2221,  0.1824,  0.1312, -0.7522, -0.3306,  0.0140, -0.0682, -0.1199,
         -0.2306, -0.4595],
        [-0.2074,  0.1492,  0.1488, -0.6946, -0.2300, -0.0601, -0.0122, -0.1334,
         -0.0079, -0.5165],
        [-0.2225,  0.1993,  0.0904, -0.7805, -0.2719, -0.0259, -0.0998, -0.0211,
          0.0227, -0.3670]], grad_fn=<AddmmBackward0>)
